# State
State是一個 字典（dict-like object），裡面存放流程運行需要的資料。
例如一個摘要 Agent 的 State：
```python
{
    "text": "原始文章內容…",
    "summary": "",
    "notes": []
}

```
每一步 Node（模型或工具）都會讀取與更新 State。

## State 的兩種寫法：TypedState vs DictState

### 方法 1：Typed State（推薦）
使用 TypedDict 或 pydantic 定義明確結構。

好處：
- 型別安全
- 方便 IDE 自動補全
- 結構清晰

In [ ]:
from typing import TypedDict, List

class ArticleState(TypedDict):
    text: str
    summary: str
    notes: List[str]

### 方法 2：Dict State
直接用 Python dict。

好處是寫起來快，但大型專案容易混亂。

In [ ]:
state = {
    "text": "",
    "summary": "",
    "notes": []
}

# Node
Node 是 LangGraph 中最重要的執行單元：

> 每個 Node 接收 state → 做某事 → 回傳要更新的 state 部分欄位。

在 LangGraph 裡：

- Node 不能直接修改整個 State
- Node 只能回傳「要更新的部分」
- Graph 會自動合併更新

In [ ]:
def summarize_node(state: ArticleState):
    summary = call_llm("請總結以下文章：" + state["text"])
    return {"summary": summary}

## Node 與 State 的互動流程
以 Node 更新 summary 為例：

### **流程**

1. Graph 執行到 summarize Node
2. 把 State 作為參數傳進去：`state`
3. Node 執行（呼叫模型或工具）
4. Node 回傳：
    
    ```python
    {"summary": "這是一段摘要"}
    
    ```
    
5. Graph 自動用回傳值更新 State

## Node 必須是純函式（pure function）：
- 不能修改原本的 State
- 不能有副作用（修改 global、寫檔案…除非必要）
- 只能讀 state → 回傳更新

## Node 更新規則
LangGraph 使用 Reducer 合併更新：

- State 是不可變（immutable）
- Node 回傳的部分 State 會 merge 進新的 State

In [ ]:
state = {"a": 1, "b": 2}

node 回傳 {"b": 3, "c": 4}

更新結果：
{"a":1, "b":3, "c":4}

## State 的更新方式
LangGraph 提供三種更新模式：

### Replace（預設）
Node 回傳值直接覆蓋同名欄位。

In [ ]:
from typing import TypedDict

class State(TypedDict):
    summary: str

In [ ]:
# 初始 State
state = {
    "summary": "舊的摘要"
}

In [ ]:
# Node 寫法
def update_summary(state: State):
    return {
        "summary": "新的摘要"
    }

In [ ]:
# 更新後 State
{
    "summary": "新的摘要"
}

### Append（列表合併）
- Node 回傳的 list → 自動 append 到原本 list

**兩種方法：**
1. add_messages（LangGraph 內建 reducer）

In [1]:
from langgraph.graph import add_messages
from typing import TypedDict, Annotated

class State(TypedDict):
    logs: Annotated[list[str], add_messages]

In [2]:
from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, START, END, add_messages

# 自動 append logs（透過 reducer）
class State(TypedDict):
    logs: Annotated[list[str], add_messages]

# 節點 Node 定義
def add_log(state: State):
    # 只回傳要新增的 logs
    return { "logs": ["取得使用者輸入"] }

# 建圖
graph_builder = StateGraph(State)
graph_builder.add_node("add_log", add_log)
graph_builder.add_edge(START, "add_log")
graph_builder.add_edge("add_log", END)

# 初始 State
initial = {"logs": ["啟動流程"]}

# 執行
result = graph_builder.compile().invoke(initial)
print(result["logs"])

[HumanMessage(content='啟動流程', additional_kwargs={}, response_metadata={}, id='769cbb8d-848b-41b5-beda-4b10b56a3311'), HumanMessage(content='取得使用者輸入', additional_kwargs={}, response_metadata={}, id='d017cd2e-160c-4764-bbdc-2aa2b18e6d7a')]


- Annotated[list[str], add_messages] 只是 metadata，Python 本身不會自動執行 add_messages。

- 只有在 LangGraph 的 StateGraph 執行節點流程時，框架才會讀取這個 reducer 並自動把新 logs append 到舊 logs。

如果不在 StateGraph 的執行流程節點執行時，會：

In [4]:
from typing import TypedDict, Annotated
from langgraph.graph import  add_messages

# 自動 append logs（透過 reducer）
class State(TypedDict):
    logs: Annotated[list[str], add_messages]

state: State = {
    "logs": ["啟動流程"]
}

# 節點 Node 定義
def add_log(state: State):
    # 只回傳要新增的 logs
    return { "logs": ["取得使用者輸入"] }

# 執行 Node
state = add_log(state)

# 輸出結果
print(state)

{'logs': ['取得使用者輸入']}


僅輸出 "取得使用者輸入" 而不是添加在原本的list

2. 使用自定義函式

In [ ]:
from typing import TypedDict, Annotated

def append_logs(old: list[str], new: list[str]) -> list[str]:
    if not new:
        return old
    return old + new

class State(TypedDict):
    logs: Annotated[list[str], append_logs]

In [3]:
from typing import TypedDict

# 合併 logs 的函數
def append_logs(old: list[str], new: list[str]) -> list[str]:
    if not new:
        return old
    return old + new

# State 定義
class State(TypedDict):
    logs: list[str]

# 初始 State
state: State = {
    "logs": ["啟動流程"]
}

# 範例 Node：加入新的 log
def add_log(state: State, new_logs: list[str]) -> State:
    # 使用 append_logs 合併
    return {
        "logs": append_logs(state["logs"], new_logs)
    }

# 測試流程
state = add_log(state, ["取得使用者輸入"])
state = add_log(state, ["驗證資料完成"])
state = add_log(state, ["流程結束"])

# 輸出結果
print(state)


{'logs': ['啟動流程', '取得使用者輸入', '驗證資料完成', '流程結束']}


### Deep Merge（巢狀結構局部更新）
- dict 內的 dict 可以「部分更新」
- 沒回傳的欄位會保留

In [ ]:
class State(TypedDict):
    user: dict

In [ ]:
# 初始 State
state = {
    "user": {
        "name": "Alice",
        "age": 30,
        "email": "alice@test.com"
    }
}

In [ ]:
# Node 只更新一個欄位
def update_age(state: State):
    return {
        "user": {
            "age": 31
        }
    }

In [ ]:
# 更新後 State
{
    "user": {
        "name": "Alice",
        "age": 31,
        "email": "alice@test.com"
    }
}
# 只改 age，其它欄位保留

# Graph
Graph 是 LangGraph 的「流程控制器」。

Graph 負責決定：
- 宣告有哪些 Node
- 定義 Node 之間的執行順序（Edge）
- 控制流程起點與終點
- 管理 State 的流動與合併

## Graph 的三個核心元素
### State
如前面提到的所示，例如：

In [ ]:
class ArticleState(TypedDict):
    text: str
    summary: str

### Node
如前面提到的所示，例如：

In [ ]:
def summarize_node(state: ArticleState):
    summary = call_llm("請總結以下文章：" + state["text"])
    return {"summary": summary}

### Edge（流程順序）
Graph 用 Edge 來描述流程：

In [ ]:
from langgraph.graph import StateGraph,START, END

graph_builder = StateGraph(State)
# 將 Node 加入 Graph
graph_builder.add_node("summarize", summarize_node)

# 建立流程
graph_builder.add_edge(START, "summarize")
graph_builder.add_edge("summarize", END)

# 編譯 graph
graph = graph_builder.compile()

意思是： START → summarize → END